In [1]:
from pamda import pamda

In [2]:
from math import log

def bmssp_big_o(n, m):
    # BMSSP has a time complexity of O(m ln(n)^(2/3))
    return m * (log(n)**(2/3))

def dijkstra_big_o(n, m):
    # Dijkstra's algorithm as implemented in scgraph has a time complexity of O((n + m) log n)
    return (m+n)*log(n)

In [3]:
benchmark_data =pamda.read_csv('./outputs/gridscale_time_tests.csv')

for row in benchmark_data:
    n_nodes = int(row['graph_nodes'])
    n_edges = int(row['graph_edges'])
    row['exp_bmssp_to_dijkstra_performance_ratio'] =  bmssp_big_o(n_nodes, n_edges) / dijkstra_big_o(n_nodes, n_edges)
    row['real_bmssp_to_dijkstra_performance_ratio'] = float(row['bmssp_spantree_time_ms']) / float(row['sc_dijkstra_spantree_time_ms'])
    row['pred_bmssp_breakeven_multiplier'] = row['exp_bmssp_to_dijkstra_performance_ratio'] / row['real_bmssp_to_dijkstra_performance_ratio']
    print(row['graph_nodes'], row['pred_bmssp_breakeven_multiplier'])

1089 0.09675224432158311
1089 0.0775117430556842
1089 0.0761341505368717
2500 0.09357434716333758
2500 0.08823097464076031
2500 0.09312258497704698
10000 0.08978226630802623
10000 0.09536512194862767
10000 0.09647588576105376
40000 0.08011298173235147
40000 0.09259871537223048
40000 0.0876048816738958
160000 0.09802650137709346
160000 0.1146749154613726
160000 0.1105968993443309
640000 0.08553491723921039
640000 0.09525847937627983
640000 0.08550098402078808
2560000 0.08674223617954024
2560000 0.09466551627534041
2560000 0.0999065744516583
10240000 0.08472541107077564
10240000 0.09502569185395306
10240000 0.08732355674850312


In [4]:

bmssp_dijkstra_big_o_ratios = []
for magnitude in list(range(1,10)) + list(range(10,300,10)):
    n = 10**magnitude
    connections_per_node = 8
    m = connections_per_node*n
    out ={
        'n_magnitude': f"1e{magnitude}",
        'connections_per_node': connections_per_node,
        'bmssp_to_dijkstra_ratio': bmssp_big_o(n,m)/dijkstra_big_o(n,m)
    }
    bmssp_dijkstra_big_o_ratios.append(out)
    print(out)

pamda.write_csv(filename='./outputs/bmssp_dijkstra_big_o_ratios.csv', data=bmssp_dijkstra_big_o_ratios)


{'n_magnitude': '1e1', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.6731454500719178}
{'n_magnitude': '1e2', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.5342758977858848}
{'n_magnitude': '1e3', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.4667329870851965}
{'n_magnitude': '1e4', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.42405506109328384}
{'n_magnitude': '1e5', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.3936578472815142}
{'n_magnitude': '1e6', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.37044621734365046}
{'n_magnitude': '1e7', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.3518921413034503}
{'n_magnitude': '1e8', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.33657272503595886}
{'n_magnitude': '1e9', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.32361457870687016}
{'n_magnitude': '1e10', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.3124464404451063}
{'n_magnitude': '1e20', '

In [5]:
from math import exp

pred_breakeven_multiplier = sum([row['pred_bmssp_breakeven_multiplier'] for row in benchmark_data]) / len(benchmark_data)
print(f"Predicted breakeven at multiplier = {pred_breakeven_multiplier}")

# TODO: Validate math here.

# Math derivation:
#
# BMSSP(n, m) = m * (ln n)^(2/3)
# Dijkstra(n, m) = (n + m) * ln n
#
# Assume m = 8n:
#   8n * (ln n)^(2/3) = p * (9n * ln n)
#
# Cancel n:
#   8 * (ln n)^(2/3) = 9p * ln n
#
# Divide by (ln n)^(2/3):
#   8 = 9p * (ln n)^(1/3)
#
# Solve for ln n:
#   (ln n)^(1/3) = 8 / (9p)
#   ln n = (8 / (9p))^3
#
# Finally, to get log10(n):
#   log10(n) = ln(n) / ln(10)

breakeven_magnitude_n = ((8 / (9 * pred_breakeven_multiplier)) ** (3)) / log(10)

print(f"Predicted breakeven at n = 1e{breakeven_magnitude_n} nodes")


Predicted breakeven at multiplier = 0.0918853158704298
Predicted breakeven at n = 1e393.17727291747116 nodes
